In [1]:
import tkinter as tk

In [2]:
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings("ignore")


In [36]:
df = pd.read_csv('ml-100k/u.data',sep='\t', names=['user_id','item_id','rating','ts'])
cols = "item_id|title| release date | video release date |\
              IMDb URL | unknown | Action | Adventure | Animation |\
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |\
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |\
              Thriller | War | Western".split('|')
movies = pd.read_csv('ml-100k/u.item',sep='|',names=cols)
data = pd.merge(df.drop('ts',axis=1),movies[['item_id','title']], on='item_id')
rate_count =  data.groupby('title')['rating'].count()
df_pivot = data.pivot_table(index='user_id',columns='title',values='rating')

app = tk.Tk()
app.geometry('500x700')
app.title('Recommendation Generator')

listbox = tk.Listbox(app, height=30, width=75, bg='pink',fg='black')
listbox.place(x=20,y=20)

for i,t in enumerate(data['title'].unique()):
    listbox.insert(i+1, t)

def recommend():
    inp = listbox.get(listbox.curselection())
    sim_df = pd.DataFrame(df_pivot.corrwith(df_pivot[inp]),columns=['corr'])
    sim_df['count'] = rate_count
    recom = sim_df[(sim_df['count']>100)].sort_values('corr',ascending=False)
    recom.drop(inp,axis=0, inplace=True)
    recommended = recom.index[:3]
    result1.set(recommended[0])
    result2.set(recommended[1])
    result3.set(recommended[2])


tk.Button(app,text='Show Recommendations', command=recommend,bg='blue',fg='white').place(x=30,y=530)

result1 = tk.Variable(app)
result1.set('.')
tk.Label(app, textvariable=result1).place(x=30,y=580)

result2 = tk.Variable(app)
result2.set('.')
tk.Label(app, textvariable=result2).place(x=30,y=620)

result3 = tk.Variable(app)
result3.set('.')
tk.Label(app, textvariable=result3).place(x=30,y=660)

app.mainloop()